## SQL ##

*En esta fase del proceso de ETL  vamos a crear paso a paso una base de datos en sql desde python, para despúes crear las tablas con sus PK y sus FK.*

*Una vez todo esto este creado procedemos a poblarla desde python con nuestro df limpio y actualizado.*

Este es el codigo usado en sql para crear nuestra base de datos. *Este codigo se puede usar para crear la base de datos directamente desde SQL,pero vamos a crearla desde este entorno*

In [1]:
import pandas as pd
import mysql.connector
import numpy as np

Este es el codigo para crear desde python la base de datos en sql.

*Correr solo una vez*

In [2]:
db = mysql.connector.connect(
     host="localhost",
     user="root",
     password="contraseña123"#Aqui teneis que poner vuestra contraeña.
)

cursor = db.cursor()

# Crear la base de datos si no existe
cursor.execute('CREATE DATABASE IF NOT EXISTS AIRBNB')

# Conectar a la base de datos recién creada
db.database = 'AIRBNB'

# Crear la tabla Hosting
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Hosting (
        record_id BIGINT,
        titles TEXT,
        property_types TEXT,
        host_name TEXT,
        PRIMARY KEY (record_id)               
    );
''')

# Crear la tabla Description_
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Description_ (
        id BIGINT,
        prices_per_night INT,
        check_in_hour TIME,
        check_out_hour TIME,
        total_hours_checkin INT,
        cleaning_fee INT,
        maximum_guests INT,
        location TEXT,
        camas INT,
        baños INT,
        dormitorios INT,
        FOREIGN KEY (id) REFERENCES Hosting(record_id)
    );
''')

# Crear la tabla Ratings
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Ratings (
        record_id BIGINT,
        ratings FLOAT,
        num_reviews FLOAT,
        FOREIGN KEY (record_id) REFERENCES Hosting(record_id)
    );
''')

#Creamos la tabla de Services_
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Services_ (
        service_id INT,
        service TEXT,
        PRIMARY KEY (service_id)
   );
''')
#Creamos la tabla de Service_ids
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Services_ids (
        service_id INT,
        record_id BIGINT,
        FOREIGN KEY (service_id) REFERENCES Services_(service_id),
        FOREIGN KEY (record_id) REFERENCES Hosting (record_id)
);
''')   
#Creamos la tabla de Category
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Category (
        category_id INT,
        category TEXT,
        PRIMARY KEY (category_id)
 );
''')

#Creamos la tabla de Category_ids
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Category_ids (
        service_id INT,
        category_id INT,
        FOREIGN KEY (service_id) REFERENCES Services_(service_id),
        FOREIGN KEY (category_id) REFERENCES Category(category_id)
);
''')   

# Cerrar el cursor y la conexión
cursor.close()
db.close()


Este es el codigo para la subida de nuestro df a la base de datos

In [2]:
df = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_final_cleaned.csv')
df_category = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_category')
df_service = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_service')
df_service_record = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_service_record')
df_category_service = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_category_service')

Dividimos las columnas del df en distintos df segun las tablas de nuestra base de datos.

*Elinamos las filas que tengan duplicada la columna record_id*

In [3]:
df_sin_duplicados = df.drop_duplicates(subset='record_id', keep=False)

In [4]:
df_sin_duplicados.head()

,urls,timestamp,record_id,titles,host_name,property_types,prices_per_night,location,ratings,num_reviews,cleaning_fee,baños,maximum_guests,dormitorios,camas,check_in_hour,check_out_hour,total_hours_checkin
18,https://www.airbnb.es/rooms/113755436612856989...,2024-11-07 10:12:02,1137554366128569898,Haz base en este alojamiento y estarás a un pa...,Steven,Habitación,40.0,"Barcelona, Catalunya, España",4.12,17.0,0.0,NaN,NaN,NaN,2.0,15:00,11:00,9.0
30,https://www.airbnb.es/rooms/124996142740169379...,2024-11-07 10:12:02,1249961427401693793,Disfruten de unos días regeneradores en un amb...,Antonio,otro,128.0,"Barcelona, Catalunya, España",5.00,4.0,0.0,1.0,2.0,2.0,2.0,NaN,11:00,NaN
36,https://www.airbnb.es/rooms/127318511126570438...,2024-11-07 10:12:02,1273185111265704381,Habitación privada para 2 huéspedes en apartam...,Mauricio,Habitación,58.0,"Barcelona, Catalunya, España",0.00,0.0,0.0,NaN,2.0,1.0,NaN,15:00,11:00,9.0
54,https://www.airbnb.es/rooms/117881510800783153...,2024-11-07 10:12:02,1178815108007831530,El apartamento ideal para sumergirte en el alm...,Lodging Apartments,Alojamiento entero,117.0,"Barcelona, Catalunya, España",0.00,2.0,95.0,1.0,2.0,1.0,1.0,15:00,NaN,9.0
57,https://www.airbnb.es/rooms/118896371995828509...,2024-11-07 10:12:02,1188963719958285093,Soy leo y quiero darte la bienvenida a una de ...,Pado,Alojamiento entero,132.0,"Barcelona, Catalunya, España",0.00,0.0,50.0,1.0,2.0,1.0,1.0,15:00,11:00,9.0


In [5]:
df_Hosting = df_sin_duplicados[['record_id','titles','property_types','host_name']]

In [6]:
df_Hosting.head()

,record_id,titles,property_types,host_name
18,1137554366128569898,Haz base en este alojamiento y estarás a un pa...,Habitación,Steven
30,1249961427401693793,Disfruten de unos días regeneradores en un amb...,otro,Antonio
36,1273185111265704381,Habitación privada para 2 huéspedes en apartam...,Habitación,Mauricio
54,1178815108007831530,El apartamento ideal para sumergirte en el alm...,Alojamiento entero,Lodging Apartments
57,1188963719958285093,Soy leo y quiero darte la bienvenida a una de ...,Alojamiento entero,Pado


In [7]:
df_service_record.head()

,service,record_id
0,1,1263117591292794971
1,2,1263117591292794971
2,3,1263117591292794971
3,4,1263117591292794971
4,5,1263117591292794971


In [8]:
df_Description_ = df_sin_duplicados[['record_id', 'prices_per_night','check_in_hour','check_out_hour','cleaning_fee','total_hours_checkin','maximum_guests','location','camas','baños','dormitorios']]
df_Description_ = df_Description_.rename(columns={'record_id': 'id'})

In [9]:
df_Ratings = df_sin_duplicados[['record_id','ratings','num_reviews']]

Aqui nos aseguramos de que no haya ningun nan sustituyendolos todos por none (valor que si admite sql).

In [10]:
df_Description_ = df_Description_.replace({np.nan : None}).reset_index(drop = True) #Sustituimos todos los posibles nan por none para poder subirlos
df_Hosting = df_Hosting.replace({np.nan : None}).reset_index(drop = True)
df_Ratings = df_Ratings.replace({np.nan : None}).reset_index(drop = True)
df_service_record = df_service_record.replace({np.nan : None}).reset_index(drop = True)
df_category_service = df_category_service.replace({np.nan : None}).reset_index(drop = True)

Aqui procedemos a la subida de las distintas tablas.

Hosting

In [12]:
column_names= ['record_id','titles','property_types','host_name']

In [13]:
database = "AIRBNB"
table_name = "Hosting"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_Hosting.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_Hosting[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Añadidas: 754 filas


Description_

In [14]:
column_names = ['id', 'prices_per_night','check_in_hour','check_out_hour','cleaning_fee','total_hours_checkin','maximum_guests','location','camas','baños','dormitorios']

In [15]:
database = "AIRBNB"
table_name = "Description_"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_Description_.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_Description_[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Añadidas: 754 filas


Ratings

In [16]:
column_names= ['record_id','ratings','num_reviews']

In [17]:
df_Ratings["record_id"] = df_Ratings["record_id"].apply(lambda x : str(x))

In [18]:
database = "AIRBNB"
table_name = "Ratings"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names
print(column_names)

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_Ratings.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"
print(insert_query)

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_Ratings[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

('record_id', 'ratings', 'num_reviews')
INSERT INTO Ratings (record_id, ratings, num_reviews) VALUES (%s, %s, %s)
Añadidas: 754 filas


Services_

In [19]:
column_names = ['service_id', 'service']

In [20]:
database = "AIRBNB"
table_name = "Services_"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_service.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_service[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Añadidas: 1143 filas


Services_ids

In [11]:
column_names = ['service_id', 'id']

In [12]:
df_service_record = df_service_record.rename(columns={'service': 'service_id'})


In [13]:
df_service_record['record_id'] = df_service_record['record_id'].astype(str)


In [14]:
database = "AIRBNB"
table_name = "Services_ids"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_service_record.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_service_record[list(column_names)].values] 

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Añadidas: 75056 filas


Category

In [ ]:
column_names = ['category_id', 'category']

In [ ]:
database = "AIRBNB"
table_name = "Category"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_category.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_category[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Añadidas: 13 filas


Category_ids

In [ ]:
column_names = ['category_id', 'service_id']

In [ ]:
df_category_service = df_category_service.rename(columns={'category': 'category_id'})
df_category_service = df_category_service.rename(columns={'service': 'service_id'})

In [ ]:
df_category_service["service_id"] = df_category_service["service_id"].apply(lambda x : str(x))

In [ ]:
df_category_service["category_id"] = df_category_service["category_id"].apply(lambda x : str(x))

In [ ]:
database = "AIRBNB"
table_name = "Category_ids"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_category_service.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_category_service[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Añadidas: 1150 filas


# Extracción de datos

In [ ]:
def extraer_datos_mysql(host, user, password, database, query):
    # Conectarse a la base de datos
    db = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    
    # Crear cursor para ejecutar la consulta
    cursor = db.cursor()

    # Ejecutar la consulta
    cursor.execute(query)
    
    # Obtener los nombres de las columnas
    column_names = cursor.column_names
    print("Nombres de las columnas:", column_names)

    # Guardar y mostrar los datos de la tabla
    data = cursor.fetchall()
    for fila in data:
        print(fila)

    # Cerrar el cursor y la conexión
    cursor.close()
    db.close()


Hosting

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Hosting;"

extraer_datos_mysql(host, user, password, database, query)

Nombres de las columnas: ('record_id', 'titles', 'property_types', 'host_name')
(80004, 'El espacioal pie del parque montjuïc, a 4 minutos del metro paralelo, este bonito y cómodo piso de tres dormitorios y un gran salón-cocina con vistas al balcon, perfectamente distribuido, satisfacerá plenamente a los visitantes más conocedores. idealmente situado en el viejo barrio barcelonés del poble sec convertido en un barrio de moda, la finca está rodeada de bares de tapas y de copas, restaurantes típicos, discotecas de moda y de teatros prestigiosos, como el apolo o el cabaret el molino inaugurado en 1898. ubicado en una calle tranquila, a un paso de la zona peatonal, el piso no está expuesto a los ruidos nocturnos. a unos pasos estan accessibles los puntos estratégicos de la ciudad : la rambla a 10 minutos, el estadio olímpico, las ferias, la fundación miró, el puerto de barcelona, el palacio güell de gaudí, la plaza de españa, el barrio gótico... el barrio está muy bien comunicado gracias a

Description_

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Description_;"

extraer_datos_mysql(host, user, password, database, query)

Nombres de las columnas: ('id', 'prices_per_night', 'check_in_hour', 'check_out_hour', 'total_hours_checkin', 'cleaning_fee', 'maximum_guests', 'location', 'camas', 'baños', 'dormitorios')
(1137554366128569898, 40, datetime.timedelta(seconds=54000), datetime.timedelta(seconds=39600), 9, 0, None, 'Barcelona, Catalunya, España', 2, None, None)
(1249961427401693793, 128, None, datetime.timedelta(seconds=39600), None, 0, 2, 'Barcelona, Catalunya, España', 2, 1, 2)
(1273185111265704381, 58, datetime.timedelta(seconds=54000), datetime.timedelta(seconds=39600), 9, 0, 2, 'Barcelona, Catalunya, España', None, None, 1)
(1178815108007831530, 117, datetime.timedelta(seconds=54000), None, 9, 95, 2, 'Barcelona, Catalunya, España', 1, 1, 1)
(1188963719958285093, 132, datetime.timedelta(seconds=54000), datetime.timedelta(seconds=39600), 9, 50, 2, 'Barcelona, Catalunya, España', 1, 1, 1)
(1100921852236177943, 113, datetime.timedelta(seconds=54000), datetime.timedelta(seconds=43200), 9, 30, 2, None, 1, 

Ratings

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Ratings;"

extraer_datos_mysql(host, user, password, database, query)

Nombres de las columnas: ('record_id', 'ratings', 'num_reviews')
(1137554366128569898, 4.12, 17.0)
(1249961427401693793, 5.0, 4.0)
(1273185111265704381, 0.0, 0.0)
(1178815108007831530, 0.0, 2.0)
(1188963719958285093, 0.0, 0.0)
(1100921852236177943, 4.71, 34.0)
(1192692130835760091, 5.0, 4.0)
(1066882264283221117, 3.63, 8.0)
(1002144028576921873, 0.0, 0.0)
(7954069, 4.67, 451.0)
(1140717328893210788, 4.48, 21.0)
(1282394462063267981, 0.0, 0.0)
(1250826161600655450, 2.86, 7.0)
(1177426002003576868, 4.69, 13.0)
(1192612229323894350, 0.0, 2.0)
(12861579, 4.9, 0.0)
(696777471141566786, 4.41, 116.0)
(1281298183182044394, 0.0, 0.0)
(1251405079105833124, 0.0, 2.0)
(1282401158185886875, 0.0, 0.0)
(1277700144392600289, 0.0, 0.0)
(1268163818980729724, 0.0, 0.0)
(38199056, 5.0, 9.0)
(1175257711239384940, 4.67, 12.0)
(49457063, 4.65, 81.0)
(1155809416104019462, 0.0, 0.0)
(31977858, 4.34, 296.0)
(24238588, 4.67, 480.0)
(33259271, 4.91, 0.0)
(1273174685854474524, 0.0, 0.0)
(1031002061578844729, 4.53,

Services

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Services_;"

extraer_datos_mysql(host, user, password, database, query)

Nombres de las columnas: ('service_id', 'service')
(1, 'secador de pelo')
(2, 'productos de limpieza')
(3, 'champú')
(4, 'gel de ducha')
(5, 'agua caliente')
(6, 'lavadora')
(7, 'perchas')
(8, 'ropa de cama')
(9, 'persianas o cortinas opacas')
(10, 'plancha')
(11, 'tendedero para ropa')
(12, 'espacio para guardar la ropa: armario')
(13, 'tv')
(14, 'aire acondicionado')
(15, 'calefacción central')
(16, 'botiquín')
(17, 'wifi')
(18, 'cocinacocina disponible para el uso de los huéspedes')
(19, 'frigorífico')
(20, 'microondas')
(21, 'congelador')
(22, 'horno')
(23, 'hervidor de agua')
(24, 'entrada independienteentrada por otra calle o edificio')
(25, 'lavandería cercana')
(26, 'patio o balcón')
(27, 'comedor al aire libre')
(28, 'ascensorel alojamiento o edificio dispone de un ascensor de al menos 132\xa0centímetros de profundidad cuya puerta mide como mínimo 81\xa0centímetros de ancho.')
(29, 'admite mascotaslos animales de asistencia siempre están permitidos')
(30, 'se permite dejar el 

Category

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Category;"

extraer_datos_mysql(host, user, password, database, query)

Nombres de las columnas: ('category_id', 'category')
(1, 'baño')
(2, 'dormitorio y lavandería')
(3, 'entretenimiento')
(4, 'calefacción y refrigeración')
(5, 'seguridad en el hogar')
(6, 'internet y oficina')
(7, 'cocina y comedor')
(8, 'características de la ubicación')
(9, 'exterior')
(10, 'aparcamiento e instalaciones')
(11, 'servicios')
(12, 'privacidad y seguridad')
(13, 'para familias')
